Let's see if we can solve a simple math problem with gpt-3.5

In [3]:
# Add prereq code to set open API key
# Load environment variables from a .env file
from dotenv import load_dotenv
load_dotenv()

import os
os.environ["OPENAI_API_KEY"] = os.environ.get('API_KEY')
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

print(os.environ["OPENAI_API_KEY"] )

voc-748426759126677430476768a3a5ba5b26b3.93141672


In [7]:
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = ChatOpenAI(model_name=model_name, temperature=temperature, max_tokens = 500)

# Note: example taken from
# https://browse.arxiv.org/pdf/2303.12712.pdf

instruction = """
Andy harvests all the tomatoes from 18 plants that have 7 tomatoes each. If he dries half the
tomatoes and turns a third of the remainder into marinara sauce, how many tomatoes are left?
"""

messages = [
    ("human", instruction),
]
print(llm.invoke(messages))

content='First, we need to find out how many tomatoes Andy harvested in total:\n18 plants * 7 tomatoes per plant = 126 tomatoes\n\nNext, we need to find out how many tomatoes Andy dried:\n126 tomatoes / 2 = 63 tomatoes dried\n\nNow, we need to find out how many tomatoes are left after drying:\n126 tomatoes - 63 tomatoes = 63 tomatoes remaining\n\nNext, we need to find out how many tomatoes Andy turned into marinara sauce:\n63 tomatoes / 3 = 21 tomatoes turned into marinara sauce\n\nFinally, we need to find out how many tomatoes are left after making marinara sauce:\n63 tomatoes - 21 tomatoes = 42 tomatoes left\n\nTherefore, Andy is left with 42 tomatoes after drying half of them and turning a third of the remainder into marinara sauce.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 167, 'prompt_tokens': 52, 'total_tokens': 219, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tok

If we run this, we see an incorrect answer:
Original Answer: 
Andy harvests a total of 18 plants * 7 tomatoes/plant = <<18*7=126>>126 tomatoes.
He dries half of the tomatoes, so he has 126 tomatoes / 2 = <<126/2=63>>63 tomatoes left.
He turns a third of the remaining tomatoes into marinara sauce, so he has 63 tomatoes * 1/3 = <<63*1/3=21>>21 tomatoes left. Answer: \boxed{21}.

In reality, Andy will have 63 - 21 = 42 tomatoes left

Let's see how we can fix this with Chain Of Thought Reasoning

First, let's setup a examples of how we want LLM to think about the problem

In [9]:

question1 = """
Karen harvests all the pears from 20 trees that have 10 pears each. She  throws a third of them away as they are rotten,
and turns a quarter of the remaining ones into jam. How many are left?
"""
answer1 = """
    First, let's calculate how many pears Gloria harvests: it's 20 * 10 = 200. 
    Then, let's calculate how many are rotten: 200 * 1/3 = 66.
    Thus, we know how many are left after she throws a third of them away: 200 - 66 = 134.
    1/4 of the remaining ones are turned into jam, or 134 * 1/4 = 33. Therefore, Karen is left with 134 - 33, or 101 pears
"""
question2 = """
Sergei harvests all the strawberries from 50 plants that have 8 stawberries each. He freezes a quarter of them,
and turns half of the remaining ones into jam. How many are left?
"""
answer2 = """
    First, let's calculate how many strawberries Sergei harvests: it's 50 * 8 = 400. 
    Then, let's calculate how many are frozen: 400 * 1/4 = 100.
    Thus, we know how many are left after he freezes 100 of them: 400 - 100 = 300.
    half of the remaining ones are turned into jam, or 300 * 1/2 = 150. Therefore, Sergei is left with 300 - 150, or 150 pears
"""

Next, let's setup an prompt template for one question/answer pair, and put our examples into an array

In [19]:
question1 = """
Karen harvests all the pears from 20 trees that have 10 pears each. She  throws a third of them away as they are rotten,
and turns a quarter of the remaining ones into jam. How many are left?
"""
answer1 = """
    First, let's calculate how many pears Gloria harvests: it's 20 * 10 = 200. 
    Then, let's calculate how many are rotten: 200 * 1/3 = 66.
    Thus, we know how many are left after she throws a third of them away: 200 - 66 = 134.
    1/4 of the remaining ones are turned into jam, or 134 * 1/4 = 33. Therefore, Karen is left with 134 - 33, or 101 pears
"""
question2 = """
Sergei harvests all the strawberries from 50 plants that have 8 stawberries each. He freezes a quarter of them,
and turns half of the remaining ones into jam. How many are left?
"""
answer2 = """
    First, let's calculate how many strawberries Sergei harvests: it's 50 * 8 = 400. 
    Then, let's calculate how many are frozen: 400 * 1/4 = 100.
    Thus, we know how many are left after he freezes 100 of them: 400 - 100 = 300.
    half of the remaining ones are turned into jam, or 300 * 1/2 = 150. Therefore, Sergei is left with 300 - 150, or 150 pears
"""


example_prompt = PromptTemplate(input_variables=["question", "answer"], 
                                template="{question}\n{answer}")
examples = [ 
    {
        "question": question1,
        "answer": answer1,
    },
    {
        "question": question2,
        "answer": answer2,
    }
]

cot_prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="Use these questions and answers to give correct response to the problem below: {input}", 
    input_variables=["input"]
)

cot_text = cot_prompt.format(input=instruction)
print("=== Chain of Thought Prompt ===")
print(cot_text)

=== Chain of Thought Prompt ===

Karen harvests all the pears from 20 trees that have 10 pears each. She  throws a third of them away as they are rotten,
and turns a quarter of the remaining ones into jam. How many are left?


    First, let's calculate how many pears Gloria harvests: it's 20 * 10 = 200. 
    Then, let's calculate how many are rotten: 200 * 1/3 = 66.
    Thus, we know how many are left after she throws a third of them away: 200 - 66 = 134.
    1/4 of the remaining ones are turned into jam, or 134 * 1/4 = 33. Therefore, Karen is left with 134 - 33, or 101 pears



Sergei harvests all the strawberries from 50 plants that have 8 stawberries each. He freezes a quarter of them,
and turns half of the remaining ones into jam. How many are left?


    First, let's calculate how many strawberries Sergei harvests: it's 50 * 8 = 400. 
    Then, let's calculate how many are frozen: 400 * 1/4 = 100.
    Thus, we know how many are left after he freezes 100 of them: 400 - 100 = 300.


Next, let's setup the FewShotPromptTemplate

Finally, let's use the same instruction as before and see if we get a different answer

In [20]:
print("=== Chain of Thought Answer ===")
print(llm.invoke([("human", cot_text)]))

=== Chain of Thought Answer ===
content="First, let's calculate how many tomatoes Andy harvests: it's 18 * 7 = 126.\nThen, let's calculate how many are dried: 126 * 1/2 = 63.\nThus, we know how many are left after he dries half of them: 126 - 63 = 63.\nA third of the remaining ones are turned into marinara sauce, or 63 * 1/3 = 21. Therefore, Andy is left with 63 - 21, or 42 tomatoes." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 391, 'total_tokens': 502, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run--ae447fac-dc2a-43fc-8056-978a565b1f37-0'
